SQLAlchemy でデータを取得する

- リレーションは `selectinload` で取得し、lazy loading はしない

In [ ]:
from database import fetch_all
from sqlalchemy import select
from sqlalchemy.orm import selectinload
from tables import ArticleTable, CommentTable, UserTable

articles = fetch_all(
    select(ArticleTable)
    .where(ArticleTable.published_at.is_not(None))
    .order_by(ArticleTable.created_at.desc())
    .options(
        selectinload(ArticleTable.author),
        selectinload(ArticleTable.comments).selectinload(CommentTable.author),
    )
)

print(articles[0].comments[1].author.name)

Carol


SQLAlchemy メモ

- SQLAlchemy クラスでオブジェクトを作成するとき (例えば `User()` と入力するとき) にフィールド名の補完が効かない
- JOIN するテーブル (上記の場合 Comment など) に対して `.where()` や `.order_by()` を指定することができない
- `.selectinload()` で eager loading したとして、ネストしている結果オブジェクトを Pydantic モデルに変換するのが簡単ではない

## Pydantic モデルへの変換

In [2]:
from database import Session, engine
from generated import Article, Comment, User
from sqlalchemy import select
from sqlalchemy.orm import selectinload
from utils import sa_to_dict

stmt = (
    select(ArticleTable)
    .where(ArticleTable.published_at.is_not(None))
    .order_by(ArticleTable.created_at.desc())
    .options(
        selectinload(ArticleTable.author),
        selectinload(ArticleTable.comments).selectinload(CommentTable.author),
    )
)

with Session(engine) as session:
    article = Article.model_validate(sa_to_dict(session.scalars(stmt).first()))

article

Article(id=UUID('c1bdbeeb-986f-41c6-8769-ee38a99a9139'), created_at=datetime.datetime(2025, 12, 26, 13, 32, 6, 551038), updated_at=datetime.datetime(2025, 12, 26, 13, 32, 6, 551038), author_id=UUID('2ac2e74d-5273-4ed1-96fa-ffc714c3ef3e'), title='Hello, SQLAlchemy 2.x', body='This is a dummy article about SQLAlchemy 2.x typed ORM.', published_at=datetime.datetime(2025, 12, 24, 13, 32, 6, 554040, tzinfo=zoneinfo.ZoneInfo(key='Etc/UTC')), author=User(id=UUID('2ac2e74d-5273-4ed1-96fa-ffc714c3ef3e'), created_at=datetime.datetime(2025, 12, 26, 13, 32, 6, 551038), updated_at=datetime.datetime(2025, 12, 26, 13, 32, 6, 551038), email='alice@example.com', name='Alice', meta={}, articles=None, comments=None), comments=[Comment(id=UUID('16b537eb-3467-4ce3-a3b1-1572f2a2f6a6'), created_at=datetime.datetime(2025, 12, 26, 13, 32, 6, 551038), updated_at=datetime.datetime(2025, 12, 26, 13, 32, 6, 551038), article_id=UUID('c1bdbeeb-986f-41c6-8769-ee38a99a9139'), author_id=UUID('d3c430d3-3ce3-45dc-9c79-1e